# Dask.distributed for parallel processing

This example demonstrates some basic techniques for parallel processing with dask.distributed. It is ad-hoc a reasonable choice for tasks that fullfill two criteria:

- Each job (for instance in a `for`-loop) is relatively slow to execute (in particular, it should take less time for the job to execute than to load the data for this job)
- The

In [1]:
from dask.distributed import Client, get_client
import numpy as np
import time
from scipy.interpolate import RegularGridInterpolator

We generate a few functions which we make artifically slow: 

In [3]:
def slow_function_for_sqrt(x: float, time_to_sleep: float =5) -> float:
    time.sleep(time_to_sleep)
    return np.sqrt(x)

def slow_function_for_add(x: float, y: float, time_to_sleep: float =5) -> float:
    time.sleep(time_to_sleep)
    return x + y

def slow_function_for_sub(x: float, y: float, time_to_sleep: float =5) -> float:
    time.sleep(time_to_sleep)
    return x - y

def slow_function_for_mul(x: float, y: float, time_to_sleep: float =5) -> float:
    time.sleep(time_to_sleep)
    return x * y

## Process jobs with `map`

In this scenario, we want to map a function over a list of inputs with dask. We can do this with the `map` method of the client. The `Client` class provides an instance of a local cluster to which jobs can be submitted. It also automatically spwans a status page which you can view in your browser. (`https://localhost:8787/status`).

In [4]:
client = Client()

The `map` function takes a function and a list of arguments. It returns a list of futures, which are objects that represent the result of the computation. The futures can be used to get the result of the computation, or to wait for the computation to finish.

In [5]:
results = client.map(slow_function_for_sqrt, [1, 2, 3])
results

[<Future: pending, key: slow_function_for_sqrt-12381480e8ae49f12c5f2b42be67d712>,
 <Future: pending, key: slow_function_for_sqrt-dbde988cb1eec861896df82ae81c9ce0>,
 <Future: pending, key: slow_function_for_sqrt-96808e77b22c7488da92918c78d425ea>]

2023-04-05 13:20:03,564 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:53964', name: 0, status: running, memory: 0, processing: 0>
2023-04-05 13:20:03,574 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:53964'.


If we check again a brief moment later, we see that the computation has finished. We can get the result of the computation with the `result` method of the future.

In [11]:
results

[<Future: finished, type: numpy.float64, key: slow_function_for_sqrt-748a9de3970cf5a3f8734bb0eb84c282>,
 <Future: finished, type: numpy.float64, key: slow_function_for_sqrt-0b277b97886bdd2aa52858d01b7cf664>,
 <Future: finished, type: numpy.float64, key: slow_function_for_sqrt-7dd7be085b4ed4d3f23afd5c7f608fa7>]

We can retrieve the results of all the futures with the `gather` method of the client. This method takes a list of futures and returns a list of results.

In [9]:
client.gather(results[0])

1.0

## Process jobs with `submit`

The key difference of using `submit` instead of `map` is that `submit` returns a future immediately, while `map` returns a list of futures. This means that we can submit jobs to the cluster before all the previous jobs have finished. This is useful if we want to submit jobs to the cluster as soon as they are available.

*Note*: Some of the results immediately show up as `finished` here: Obviously, `dask` detects that we are using the same function and arguments multiple times and only submits the job once.

In [16]:
results = []
for x in range(10):
    args = [1, 2]
    results.append(client.submit(slow_function_for_add, 1, 2))
results

[<Future: pending, key: slow_function_for_add-efd93f208a3d2d884a11fe7c29c141c2>,
 <Future: finished, type: int, key: slow_function_for_add-993e71a80c0c1a0a04d9379873ea2a45>,
 <Future: pending, key: slow_function_for_add-606c2b0346b78467733f1c98bba3f521>,
 <Future: pending, key: slow_function_for_add-a92aea6c9ca04ffbdd382f853fcc6a4f>,
 <Future: pending, key: slow_function_for_add-31f89d276560bb2908c140c9dd70c5da>,
 <Future: pending, key: slow_function_for_add-bcf96c99db1b76ca30682af7c92ac7b1>,
 <Future: pending, key: slow_function_for_add-da6e427be686a6fed9dbebbe3483a189>,
 <Future: pending, key: slow_function_for_add-55a7ef39f26e7d745823fb7eebdea4e3>,
 <Future: pending, key: slow_function_for_add-fb1a1e13095b4dc997832f7625c3a309>,
 <Future: pending, key: slow_function_for_add-978b65963b046f4347eb13ab275219f4>]

This is super helpful if we want to pass the results of one computation to another computation. For example, we can use the results of the previous computation to compute the sum of the squares of the numbers.

In [19]:
results = []
for x in range(10):
    
    # calculate square root and square
    square_root = client.submit(slow_function_for_sqrt, x)
    square = client.submit(slow_function_for_mul, x, x)

    # calculate square root + square
    results.append(client.submit(slow_function_for_add, square_root, square))
results

[<Future: pending, key: slow_function_for_add-5fdbe2ebb5dc9beb4255df120b7a9f8b>,
 <Future: pending, key: slow_function_for_add-cbdcd20327d9709495f7ade05dab25e4>,
 <Future: pending, key: slow_function_for_add-71a175d0e38f3d291166f9f93d026985>,
 <Future: pending, key: slow_function_for_add-da823d26aed51540597a5e2a940167c4>,
 <Future: pending, key: slow_function_for_add-442f431601a6d1ff291046a5ebb9424c>,
 <Future: pending, key: slow_function_for_add-6c8ddf9f50d1263ab6da300fbf454e63>,
 <Future: pending, key: slow_function_for_add-608d77076418f2381d0a571fec4c98f0>,
 <Future: pending, key: slow_function_for_add-30e371cb2c84c4377419eceb803f1080>,
 <Future: pending, key: slow_function_for_add-7bbf3e04726f2923f4c879cea613e692>,
 <Future: pending, key: slow_function_for_add-81dc629431f575e59c818cc341e15dba>]

## Catching errors

One intriguing feature of using dask to process big amounts of data, is that we can catch errors and continue processing the rest of the data. We can demonstrate this by passing some negative values to the functions we created above, among which the `sqrt` function will raise an error. To demonstrate this, we have to modify one of the functions above:

In [20]:
def slow_function_for_sqrt(x: float, time_to_sleep: float =5) -> float:
    time.sleep(time_to_sleep)
    if x < 0:
        raise ValueError("x must be positive")
    return np.sqrt(x)

In [21]:
results = []
for x in range(-2, 10):
    
    # calculate square root and square
    square_root = client.submit(slow_function_for_sqrt, x)
    square = client.submit(slow_function_for_mul, x, x)

    # calculate square root + square
    results.append(client.submit(slow_function_for_add, square_root, square))

In [14]:
results

[<Future: error, key: slow_function_for_add-d15ecd5f3e59bc81c64e8d8bd54bfcf5>,
 <Future: error, key: slow_function_for_add-f0e21b48504a6b412d640d6a046cbb31>,
 <Future: finished, type: numpy.float64, key: slow_function_for_add-757441b4c19ca7a4671e18f2f7804c27>,
 <Future: finished, type: numpy.float64, key: slow_function_for_add-de5787ab9e2771dec32a7e1bfe35a34d>,
 <Future: finished, type: numpy.float64, key: slow_function_for_add-f1f95fa54c21489b9c10157f2fbd61d8>,
 <Future: finished, type: numpy.float64, key: slow_function_for_add-508a80244946e066568a369448c355cc>,
 <Future: finished, type: numpy.float64, key: slow_function_for_add-048b854e8162512061216a24f7c41232>,
 <Future: finished, type: numpy.float64, key: slow_function_for_add-db8b30ca43a8ccfd9352c69db8672089>,
 <Future: finished, type: numpy.float64, key: slow_function_for_add-5a513bf5b05ac474b1f44eb84d627c82>,
 <Future: finished, type: numpy.float64, key: slow_function_for_add-7eea42c67d1f806d5a8fc9496b445951>,
 <Future: finished

## Process functions with multiple arguments

We can also process functions with multiple arguments with `map` and `submit`. To demonstrate this in a more practical scenario, we show how to interpolate an image with `scipy.interpolate.RegularGridInterpolator`. We first create a function which takes a list of coordinates and returns the interpolated values at those coordinates.

In [22]:
def interpolate_at_point(image: np.ndarray, coordinate: np.ndarray, interpolation_method='nearest') -> float:
    interpolator = RegularGridInterpolator((np.arange(1000), np.arange(1000)), image, method=interpolation_method)
    time.sleep(2)
    return interpolator(coordinate)

We also create some radom data: A `1000x1000` image with random values between `0` and `1` and a list of `300` random coordinates at which we want to interpolate the image.

In [ ]:
image = np.random.random(size=(1000, 1000))
sampling_coordinates = np.random.random(size=(300, 2)) * 1000
sampling_coordinates

With `client.submit`, we can submit the interpolation job to the cluster. To each call, we can pass a modified set of arguments and keyword arguments using the splash-operator (`*` and `**`):

`client.submit(function, *arguments, **keyword_arguments)`

In [24]:
results = []
for idx, coordinate in enumerate(sampling_coordinates):
    arguments = (image, coordinate)
    
    # use different interpolation methods for every second point
    if idx % 2 == 0:
        keyword_arguments = {'interpolation_method': 'nearest'}
    else:
        keyword_arguments = {'interpolation_method': 'linear'}

    results.append(client.submit(interpolate_at_point, *arguments, **keyword_arguments))

C:\Users\johamuel\Anaconda3\envs\stress\lib\site-packages\distributed\worker.py:3039: UserWarning: Large object of size 7.63 MiB detected in task graph: 
  [array([[0.0306258 , 0.86577458, 0.94539521, ...,  ... 658.64235219])]
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


We can also use `client.map` to submit jobs with multiple arguments. Note that we don't need to repeat the keyword arguments n times; We can spceify them once and they will be passed to each call of the function.

In [31]:
# Copy the arguments and keyword arguments n times:
arguments = ([image] * len(sampling_coordinates), sampling_coordinates)
keyword_arguments = {'interpolation_method': 'nearest'}

# Submit the job
results = client.map(interpolate_at_point,  *arguments, **keyword_arguments)
results = client.gather(results)

## Scattering data

For this, we receive warnings that we should use `client.scatter`. This is the case if the data we want to pass to every job is relatively "large". We can use `client.scatter` to distribute the data to the workers rather than putting it into our local memory. This method returns a list of futures, which we can use to get the distributed data. The key difference between using `.scatter` or avoiding this is the following:

* With `scatter`: The data is distributed to the workers and the workers can access the data directly.
* Without `scatter`: The data is first loaded into the local memory of the client and afterwards distributed to the workers. Hence, the data needs to be loaded twice.

In [36]:
results = []
image_future = client.scatter(image)
for coordinate in sampling_coordinates:
    results.append(client.submit(interpolate_at_point, image_future, coordinate))

## Sending jobs to a client from within a function

In some scenarios, we want to send jobs to a worker from a namespace where we do not have access to the `client`. In this case, we can retrieve the `client` with the `get_client` function (Example taken from [here](https://distributed.dask.org/en/stable/task-launch.html?highlight=delayed#dask-delayed))

In [20]:
def fibonacci(number_of_iterations: int):
    if number_of_iterations < 2:
        return number_of_iterations
    client = get_client()
    a_future = client.submit(fibonacci, number_of_iterations - 1)
    b_future = client.submit(fibonacci, number_of_iterations - 2)
    a, b = client.gather([a_future, b_future])
    time.sleep(5)
    return a + b

In [21]:
future = client.submit(fibonacci, 10)

In [ ]:
client.gather(future)

55

2023-03-13 21:40:03,965 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:57385', name: 1, status: running, memory: 82, processing: 0>
2023-03-13 21:40:03,990 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:57393', name: 0, status: running, memory: 85, processing: 8>
2023-03-13 21:40:04,014 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:57393'.
2023-03-13 21:40:04,017 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:57385'.
2023-03-13 21:40:06,437 - distributed.nanny - WARNING - Restarting worker
2023-03-13 21:40:06,441 - distributed.nanny - WARNING - Restarting worker
